<a href="https://colab.research.google.com/github/harsha-9977/DSA/blob/main/Day45_Stack%26Queue2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Next Smaller Element
def nextSmaller(arr):
  stack = []
  result = [-1] * len(arr)

  for i in range(len(arr)-1, -1, -1):
    while stack and stack[-1] >= arr[i]:
      stack.pop()

    if stack:
      result[i] = stack[-1]

    stack.append(arr[i])

  return result
print(nextSmaller([4, 8, 5, 2, 25]))

[2, 5, 2, -1, -1]


In [ ]:
# Least Recently Used !!!IMPORTANT
class Node:
  def __init__(self, key, val):
    self.key = key
    self.val = val
    self.prev = None
    self.next = None

class LRUCache:
  def __init__(self, capacity):
    self.cap = capacity
    self.cache = {}
    self.left = Node(0,0)
    self.right = Node(0,0)
    self.left.next = self.right
    self.right.prev = self.left

  def put(self, key, value):
    if key in self.cache:
      self._remove(self.cache[key])
    self.cache[key] = Node(key,value)
    self._insert(self.cache[key])

    if len(self.cache) > self.cap:
      lru = self.left.next
      self._remove(lru)
      del self.cache[lru.key]

  def get(self, key):
    if key in self.cache:
      self._remove(self.cache[key])
      self._insert(self.cache[key])
      return self.cache[key].val
    return -1

  def _remove(self,node):
    prev, nxt = node.prev, node.next
    prev.next = nxt
    nxt.prev = prev

  def _insert(self,node):
    prev, nxt = self.right.prev, self.right
    prev.next = node
    nxt.prev = node
    node.next = nxt
    node.prev = prev

inputs = ["LRUCache", "put", "put", "get", "put", "get", "put", "get", "get", "get"]
params = [[2], [1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]

output = []
lru = None

for i in range(len(inputs)):
    if inputs[i] == "LRUCache":
        lru = LRUCache(*params[i])
        output.append(None)
    elif inputs[i] == "put":
        lru.put(*params[i])
        output.append(None)
    elif inputs[i] == "get":
        res = lru.get(*params[i])
        output.append(res)

print("Output:", output)


Output: [None, None, None, 1, None, -1, None, -1, 3, 4]


In [ ]:
# Most Recently Used

class Node:
  def __init__(self, key, val):
    self.key = key
    self.val = val
    self.prev = None
    self.next = None

class MRUCache:
  def __init__(self, capacity):
    self.cap = capacity
    self.cache = {}
    self.left = Node(0,0)
    self.right = Node(0,0)
    self.left.next = self.right
    self.right.prev = self.left

  def put(self, key, val):
    # If the key exists, remove it first (will re-insert as MRU)
    if key in self.cache:
        self._remove(self.cache[key])
        del self.cache[key]
    # If the cache is full, evict MRU (right before tail)
    if len(self.cache) >= self.cap:
        mru = self.right.prev
        self._remove(mru)
        del self.cache[mru.key]
    # Now, insert the new node as MRU
    new_node = Node(key, val)
    self._insert(new_node)
    self.cache[key] = new_node


  def get(self, key):
      if key in self.cache:
        self._remove(self.cache[key])
        self._insert(self.cache[key])
        return self.cache[key].val
      return -1

  def _remove(self, node):
      prev, nxt = node.prev, node.next
      prev.next = nxt
      nxt.prev = prev

  def _insert(self, node):
      prev, nxt = self.right.prev, self.right
      prev.next = node
      nxt.prev = node
      node.next = nxt
      node.prev = prev


inputs = ["MRUCache", "put", "put", "get", "put", "get", "put", "get", "get", "get"]
params = [[2], [1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]

output = []
mru = None

for i in range(len(inputs)):
    if inputs[i] == "MRUCache":
        mru = MRUCache(*params[i])
        output.append(None)
    elif inputs[i] == "put":
        mru.put(*params[i])
        output.append(None)
    elif inputs[i] == "get":
        res = mru.get(*params[i])
        output.append(res)

print("Output:", output)

Output: [None, None, None, 1, None, 2, None, -1, 3, 4]


In [ ]:
# LFU cache Least Frequently Used

from collections import defaultdict, OrderedDict

class LFUCache:
  def __init__(self, capacity):
    # Max book our shelf can hold
    self.cap = capacity
    # store book with content
    self.key_to_val = {}
    # store book with borrowed (popularity)
    self.key_to_freq = {}
    # GROUP the books with its popularity
    self.freq_to_keys = defaultdict(OrderedDict)
    # which is current least popular
    self.min_freq = 0

  def _update_freq(self, key):
    # check the current popularity
    freq = self.key_to_freq[key]
    # remove BOOK from old popularity GROUP
    del self.freq_to_keys[freq][key]
    # after removing book if old popularity GROUP is empty
    if not self.freq_to_keys[freq]:
      # delete that group
      del self.freq_to_keys[freq]
      # check whether this group was least popular
      if self.min_freq == freq:
        # if yes increment by 1
        self.min_freq += 1

    # add book to the new popularity group
    new_freq = freq + 1
    self.freq_to_keys[new_freq][key] = None
    # update book popularity count
    self.key_to_freq[key] = new_freq

  def get(self, key):
    # if book is not in shelf cant borrow
    if key not in self.key_to_val:
      return -1
    # borrowing this book makes it popular
    self._update_freq(key)

    # return book content
    return self.key_to_val[key]

  def put(self, key, val):
    # no shelf capacity, Do Nothing
    if self.cap == 0:
      return

    # update the content of the book if its already on the shelf
    if key in self.key_to_val:
      self.key_to_val[key] = val
      # also make it popular
      self._update_freq(key)
      # return
      return

    # if shelf capacity is full, remove least popular and oldest borrowed book
    if len(self.key_to_val) >= self.cap:
      lfu_key, _ = self.freq_to_keys[self.min_freq].popitem(last=False)
      del self.key_to_val[lfu_key]
      del self.key_to_freq[lfu_key]
      if not self.freq_to_keys[self.min_freq]:
        del self.freq_to_keys[self.min_freq]

    # add new BOOK with popularity 1
    self.key_to_val[key] = val
    self.key_to_freq[key] = 1
    self.freq_to_keys[1][key] = None
    self.min_freq = 1

inputs = ["LFUCache", "put", "put", "get", "put", "get", "get", "put", "get", "get", "get"]
params = [[2], [1, 1], [2, 2], [1], [3, 3], [2], [3], [4, 4], [1], [3], [4]]

output = []
lfu = None

for i, command in enumerate(inputs):
    if command == "LFUCache":
        lfu = LFUCache(*params[i])
        output.append(None)
    elif command == "put":
        lfu.put(*params[i])
        output.append(None)
    elif command == "get":
        res = lfu.get(*params[i])
        output.append(res)

print(output)


[None, None, None, 1, None, -1, 3, None, -1, 3, 4]


In [1]:
# Max of Min for every window size
def maxOfmin(arr):
  n = len(arr)
  res = [0] * (n+1)

  left = [-1] * n
  right = [n] * n

  # left previous small
  stack = []
  for i in range(n):
    while stack and arr[stack[-1]] >= arr[i]:
      stack.pop()
    left[i] = stack[-1] if stack else -1
    stack.append(i)

  # right previous small
  stack = []
  for i in reversed(range(n)):
    while stack and arr[stack[-1]] >= arr[i]:
      stack.pop()
    right[i] = stack[-1] if stack else -1
    stack.append(i)

  # get length and compare with each arr element
  for i in range(n):
    length = right[i] - left[i] - 1
    res[length] = max(res[length], arr[i])


  # clean up
  for i in range(n-1,0,-1):
    res[i] = max(res[i], res[i+1])

  return res[1:]
print(maxOfmin([10, 20, 30, 50, 10, 70, 30]))

[70, 30, 20, 10, 10, 10, 10]


In [3]:
# Find Celebraty
def celeb(mat):
  n = len(mat)
  cand = 0

  for i in range(1,n):
    if mat[cand][i] == 1:
      cand = i

  for i in range(n):
    if cand != i and (mat[cand][i] == 1 or mat[i][cand] == 0):
      return -1
  return cand
print(celeb([[1, 1, 0], [0, 1, 0], [0, 1, 1]]))

1
